---

<center><b> Developed Code<br> </b>Developed Code에서는 재현 코드 Class 일부를 수정하고, 추가 분석을 통해 발견하였던 유의미한 파생변수를 Concat하여 데이터를 생성하였습니다.<br>이후 똑같은 Dual Blending(CatBoostRegressor)를 이용하여 Submission.csv를 생성하였습니다.<br></center>

---

# <center>1. 데이터셋 준비</center>

---

## 1-1. 라이브러리 / 데이터 로드

In [1]:
from tqdm import tqdm

import pandas as pd
import numpy as np

import datetime as dt
import collections
import random
from catboost import CatBoostRegressor

from sklearn.metrics import *
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings(action='ignore') 

DATA_PATH = "./preprocessing/"
SAVE_PATH = "./preprocessing/"
RESULT_PATH = "./result(developed)/"

In [2]:
# 데이터 로드
train_err = pd.read_csv(DATA_PATH + 'train_err_data_time.csv')
train_qual = pd.read_csv(DATA_PATH + "train_quality_data_time.csv")
train_problem = pd.read_csv(DATA_PATH + 'train_problem_data_time.csv')

test_err = pd.read_csv(DATA_PATH + "test_err_data_time.csv")
test_qual = pd.read_csv(DATA_PATH + 'test_quality_data_time.csv')

# 불만 접수 고객 확인
problem_user = train_problem.user_id.unique()
no_problem_user = list(set(train_err.user_id.unique())  - set(problem_user))

In [3]:
# 난수 고정
random.seed(1422)

In [4]:
def time2int(x):
    x = x[:4] + x[5:7] + x[8:10] + x[11:13] + x[14:16] + x[17:]
    x = int(x)
    return x

In [5]:
train_err.time = train_err.time.apply(lambda x : time2int(x))
train_qual.time = train_qual.time.apply(lambda x : time2int(x))
train_problem.time = train_problem.time.apply(lambda x : time2int(x))

test_err.time = test_err.time.apply(lambda x : time2int(x))
test_qual.time = test_qual.time.apply(lambda x : time2int(x))

---

# <center>2. 데이터 전처리</center>

---

## 2-1. 변수 처리

In [6]:
# 데이터 기본 정보
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [7]:
class pre_funcs() : # 데이터 전처리 함수들
    
    def make_datetime(x): # datetime 데이터로 변환
        x     = str(x)
        year  = int(x[:4])
        month = int(x[4:6])
        day   = int(x[6:8])
        hour  = int(x[8:10])
        minute  = int(x[10:12])
        sec  = int(x[12:])
        return dt.datetime(year, month, day, hour,minute,sec)
    
    def get_total(col,df1,df2): # 각 데이터별 모든 종류구하기
        train_unique = set(df1[col].unique())
        test_unique  = set(df2[col].unique())
        total = (train_unique | test_unique) 
        return total
    
    def make_day(col): # 일자 반환
        col = str(col)
        year  = int(col[:4])
        month = int(col[4:6])
        day   = int(col[6:8])

        if month == 10:
            return 0
        elif month == 11:
            return day
        else:
            return 32
    
    def make_hour(col): # 시 반환
        return int(str(col)[8:10])
    
    def str2int(x): # string -> int
        if type(x) == str:
            x = x.replace(",","")
            x = float(x)
            return int(x)
        else:
            return int(x)

In [8]:
# 파생변수 목적에 맞게 각기 다른 전처리 : err1 - mk_dataset1 용
train_err1 = train_err.copy()
train_err1['date_k'] = train_err1['time'].apply(lambda x:pre_funcs.make_day(x))
train_err1['hour'] = train_err1['time'].apply(lambda x:pre_funcs.make_hour(x))

test_err1 = test_err.copy()
test_err1['date_k'] = test_err1['time'].apply(lambda x:pre_funcs.make_day(x))
test_err1['hour'] = test_err1['time'].apply(lambda x:pre_funcs.make_hour(x))

In [9]:
# 파생변수 목적에 맞게 각기 다른 전처리 : err2 - mk_dataset3 용
train_err2 = train_err.copy()
test_err2 = test_err.copy()

train_err2.time = train_err2.time.apply(lambda x: pre_funcs.make_datetime(x))
test_err2.time = test_err2.time.apply(lambda x: pre_funcs.make_datetime(x))

model_total= pre_funcs.get_total('model_nm',train_err2,test_err2)
errtype_total=pre_funcs.get_total('errtype',train_err2,test_err2)
fwver_total = pre_funcs.get_total('fwver',train_err2,test_err2) 

In [10]:
# quality 
# qual1 : train / qual2 : test
qual1 = train_qual.copy()
qual1['date'] = qual1['time'].apply(lambda x: int(str(x)[:8]))
list_qual = list(qual1.columns[3:-1])
for i in list_qual:
    qual1[i] = qual1[i].apply(lambda x: pre_funcs.str2int(x))
    

qual2 = test_qual.copy()
qual2['date'] = qual2['time'].apply(lambda x: int(str(x)[:8]))
list_qual2 = list(qual2.columns[3:-1])
for i in list_qual:
    qual2[i] = qual2[i].apply(lambda x: pre_funcs.str2int(x))

## 2-2. 파생변수 생성

### 2-2-1. make_dataset1

In [11]:
class make_dataset1 : # 에러타입/ 에러코드 관련 파생변수 생성 
    
    def __init__(self, df, types, num_user, min_idx):
        self.df = df
        self.types = types
        self.num_user = num_user
        self.min_idx = min_idx
        
    def run(self) : # 전체 실행 함수
        self.critical_errcode = self.make_errcode_list()
        result = self.df_make()
        return result
        
    def make_errcode_list (self): # 주요 에러코드 리스트 생성 함수
        label = np.zeros(15000)
        label[train_problem.user_id.unique()-10000] = 1 

        problem_list = list(problem_user)
        problem_code= train_err[train_err['user_id'].isin(problem_list)]['errcode'].value_counts().to_dict()
        no_problem_code = train_err[~train_err['user_id'].isin(problem_list)]['errcode'].value_counts().to_dict()

        total_code = []
        critical_errcode = []
        for i in problem_code:
            try :
                total_code.append((i,problem_code[i]/(problem_code[i]+no_problem_code[i]),problem_code[i],no_problem_code[i]))
            except :
                total_code.append((i,1,problem_code[i],0))

        for i in no_problem_code:
            if i in problem_code:
                continue
            total_code.append((i,0,0,no_problem_code[i]))

        total_code.sort(key = lambda x: x[1], reverse = True)


        for i in total_code:
            if i[2] >= 5 or i[3] >= 5:
                critical_errcode.append(i[0])
        return critical_errcode

    def df_make (self) : # 에러코드, 날짜, 시간대 데이터 생성 함수
        num_user = self.num_user
        min_idx = self.min_idx
        clist = self.critical_errcode

        id_error = self.df[['user_id','errtype','errcode', 'fwver', 'date_k', 'hour']].values
        error = np.zeros((num_user,223))
        vers = [' '] * num_user

        # 0~ 41 : 에러타입, 42~165: 에러코드분류, 166~198: 날짜, 199~222 : 시간대
        for person_idx, err, code, v, d, h in tqdm(id_error):
            error[person_idx - min_idx,err - 1] += 1

            if code in clist :
                error[person_idx - min_idx,42+clist.index(code)] += 1

            vers[person_idx - min_idx] = v

            error[person_idx - min_idx,166 + d ] += 1
            error[person_idx - min_idx,199 + h ] += 1

        res = pd.DataFrame(data=error, columns = 
                ['err_' + str(i+1) + '_cnt' for i in range(42)]+
                ['err_' + str(i+1) + '_code' for i in range(124)]+
                ['err_' + str(i+1) + '_date' for i in range(33)]+
                ['err_' + str(i+1) + '_hour' for i in range(24)])
        res['Total'] = res.sum(axis=1)
        res['ver'] = vers
        return res

In [12]:
train1 = make_dataset1(train_err1,'train', 15000, 10000).run()
test1 = make_dataset1(test_err1,'test', 14999, 30000).run()

100%|██████████| 16532649/16532649 [01:09<00:00, 238442.90it/s]


### 2-2-2. make_dataset2

In [13]:
class make_dataset2():# 에러타입 파생변수, 퀄리티 파생변수 생성
    def __init__(self, df, scope, num_user, min_idx):
        self.df = df
        self.scope = scope
        self.num_user = num_user
        self.min_idx = min_idx
    
    def make_qual_data(self): # 퀄리티 관련 함수 실행
    
        qual_tmp0 = self.move_quality(self.df, 0)
        qual_tmp1 = self.move_quality(self.df, 1)
        qual_tmp2 = self.move_quality(self.df, 2)
        qual_tmp3 = self.move_quality(self.df, 3)
        qual_tmp4 = self.move_quality(self.df, 4)
        qual_tmp5 = self.move_quality(self.df, 5)
        qual_tmp6 = self.move_quality(self.df, 6)
        qual_tmp7 = self.move_quality(self.df, 7)
        qual_tmp8 = self.move_quality(self.df, 8)
        qual_tmp9 = self.move_quality(self.df, 9)
        qual_tmp10 = self.move_quality(self.df, 10)
        qual_tmp11 = self.move_quality(self.df, 11)
        qual_tmp12 = self.move_quality(self.df, 12)
        
        qual_plus = self.qual_cnt()
        
        return pd.concat([qual_plus, qual_tmp0, qual_tmp1, qual_tmp2, qual_tmp3, 
                        qual_tmp4, qual_tmp5, qual_tmp6, qual_tmp7, qual_tmp8, qual_tmp9, qual_tmp10,
                        qual_tmp11, qual_tmp12], axis=1).fillna(0)
    
    
    def move_errtype(self): # 단위 시간별 에러타입 통계수치
        tqdm.pandas()
        data = self.df
        data2 = data.copy()
        scope= self.scope
        data2['date'] = data['time'].apply(lambda x: int(str(x)[:8]))

        # user_id, date, errtype
        errtype_array = np.zeros((self.num_user, 30, 42))
        user_id_min = min(data2['user_id'])

        def myfunc(x):
            for day in range(x['date'] % 100, min(31, x['date'] % 100 + scope)):
                errtype_array[x['user_id'] - user_id_min, day-1, x['errtype']-1] += 1

        data2.progress_apply(myfunc, axis=1)

        errtype_array_max = errtype_array.max(axis=1)
        errtype_array_min = errtype_array.min(axis=1)
        errtype_array_std = errtype_array.std(axis=1)

        return pd.concat([pd.DataFrame(data=errtype_array_max, columns=['err_' + str(i+1) + 
                                                                        '_move_' + str(scope) + 
                                                                        '_max' for i in range(42)]),
                          pd.DataFrame(data=errtype_array_min, columns=['err_' + str(i+1) + 
                                                                        '_move_' + str(scope) + 
                                                                        '_min' for i in range(42)]), 
                          pd.DataFrame(data=(errtype_array_max - errtype_array_min), columns=['err_' + str(i+1) + 
                                                                                              '_move_' + str(scope) + 
                                                                                              '_diff' for i in range(42)]), 
                          pd.DataFrame(data=errtype_array_std, columns=['err_' + str(i+1) + 
                                                                        '_move_' + str(scope) + 
                                                                        '_std' for i in range(42)])], axis=1)
    def qual_cnt(self) : # 퀄리티 단순합
        id_q = self.df[['user_id','fwver',"quality_0","quality_1","quality_2","quality_3","quality_4","quality_5","quality_6","quality_7","quality_8","quality_9","quality_10","quality_11","quality_12"]].values
        qual = np.zeros((self.num_user,41))
        
        # 0~ 27 : 펌웨어, 28~40: 퀄리티
        for person_idx, fv, q0, q1, q2, q3, q4, q5, q6, q7, q8, q9, q10, q11, q12 in tqdm(id_q):

            qual[person_idx - self.min_idx,28] += q0
            qual[person_idx - self.min_idx,29] += q1
            qual[person_idx - self.min_idx,30] += q2
            qual[person_idx - self.min_idx,31] += q3
            qual[person_idx - self.min_idx,32] += q4
            qual[person_idx - self.min_idx,33] += q5
            qual[person_idx - self.min_idx,34] += q6
            qual[person_idx - self.min_idx,35] += q7
            qual[person_idx - self.min_idx,36] += q8
            qual[person_idx - self.min_idx,37] += q9
            qual[person_idx - self.min_idx,38] += q10
            qual[person_idx - self.min_idx,39] += q11
            qual[person_idx - self.min_idx,40] += q12
            
        qual_plus = pd.DataFrame(data=qual, columns = ['err_'+str(i+1)+'_qual' for i in range(41)])
        qual_plus = qual_plus.iloc[:,28:]
        qual_plus.columns = ['err_'+str(i)+'_qual' for i in range(13)]
        return qual_plus
    
    def make_id_in_qual(self): # 퀄리티 데이터에 없는 id 채우기
        data = self.df
        data2 = data.copy()
        lst_id1 = sorted(list(data2.user_id.unique()))

        for i in tqdm(range(self.min_idx, self.num_user +self.min_idx)):
            if i in lst_id1:
                pass
            elif i not in lst_id1:
                tmp_d = {'time': 20201101090000,
                         'user_id': i,
                         'fwver': '04.11.1384',
                         'quality_0':0,'quality_1':0,'quality_2':0,'quality_3':0,'quality_4':0,'quality_5':0,
                         'quality_6':0,'quality_7':0,'quality_8':0,'quality_9':0,'quality_10':0,'quality_11':0,
                         'quality_12':0}
                data2 = data2.append(tmp_d, ignore_index=True)

        return data2
    
    def move_quality(self, data, number): # 퀄리티 통계수치
        tqdm.pandas()
        data2 = data.copy()
        ap_data_tr = pd.DataFrame({"user_id" : np.arange(self.min_idx, self.min_idx + self.num_user)})

        # -1값 0으로 대체
        for i in data2.columns[3:]:
            idx = data2[data2[i] == -1].index
            data2.loc[idx, i] = 0


        # -----------------------------------------------------------------------------------------------------
        # 이동평균의 평균
        mean_data = data2.groupby(['user_id','time'])['quality_' + str(number)].rolling(window=12).mean().\
        to_frame().reset_index().dropna()
        del mean_data['level_2']
        mean_data = mean_data.groupby('user_id')['quality_' + str(number)].mean().to_frame().reset_index()
        mean_data = pd.merge(ap_data_tr, mean_data, how='left', on = 'user_id').fillna(0)
        mean_data = mean_data.iloc[:,1]


        # 이동최대값의 최대값
        max_data = data2.groupby(['user_id','time'])['quality_' + str(number)].rolling(window=12).max().\
        to_frame().reset_index().dropna()
        del max_data['level_2']
        max_data = max_data.groupby('user_id')['quality_' + str(number)].max().to_frame().reset_index()
        max_data = pd.merge(ap_data_tr, max_data, how='left', on = 'user_id').fillna(0)
        max_data = max_data.iloc[:,1]


        # 이동최소값의 최소값
        min_data = data2.groupby(['user_id','time'])['quality_' + str(number)].rolling(window=12).min().\
        to_frame().reset_index().dropna()
        del min_data['level_2']
        min_data = min_data.groupby('user_id')['quality_' + str(number)].min().to_frame().reset_index()
        min_data = pd.merge(ap_data_tr, min_data, how='left', on = 'user_id').fillna(0)
        min_data = min_data.iloc[:,1]


        # 이동표준편차의 평균
        std_data = data2.groupby(['user_id','time'])['quality_' + str(number)].rolling(window=12).std().\
        to_frame().reset_index().dropna()
        del std_data['level_2']
        std_data = std_data.groupby('user_id')['quality_' + str(number)].mean().to_frame().reset_index()
        std_data = pd.merge(ap_data_tr, std_data, how='left', on = 'user_id').fillna(0)
        std_data = std_data.iloc[:,1]


        # 이동평균의 최대값
        meanmax_data = data2.groupby(['user_id','time'])['quality_' + str(number)].rolling(window=12).mean().\
        to_frame().reset_index().dropna()
        del meanmax_data['level_2']
        meanmax_data = meanmax_data.groupby('user_id')['quality_' + str(number)].max().to_frame().reset_index()
        meanmax_data = pd.merge(ap_data_tr, meanmax_data, how='left', on = 'user_id').fillna(0)
        meanmax_data = meanmax_data.iloc[:,1]


        # 이동최대값의 평균
        maxmean_data = data2.groupby(['user_id','time'])['quality_' + str(number)].rolling(window=12).max().\
        to_frame().reset_index().dropna()
        del maxmean_data['level_2']
        maxmean_data = maxmean_data.groupby('user_id')['quality_' + str(number)].mean().to_frame().reset_index()
        maxmean_data = pd.merge(ap_data_tr, maxmean_data, how='left', on = 'user_id').fillna(0)
        maxmean_data = maxmean_data.iloc[:,1]


        # merge
        moving_result = pd.concat([mean_data, max_data, min_data, std_data, meanmax_data, maxmean_data], axis=1)

        moving_result.columns = ['quality_'+str(number)+'_mean', 'quality_'+str(number)+'_max',
                                 'quality_'+str(number)+'_min', 'quality_'+str(number)+'_std',
                                 'quality_'+str(number)+'_meanmax', 'quality_'+str(number)+'_maxmean']

        return moving_result
    

In [14]:
train2 = make_dataset2(train_err,3,15000,10000).move_errtype()
train3 = make_dataset2(train_err,7,15000,10000).move_errtype()
train4 = make_dataset2(qual1,7,15000,10000).make_qual_data()

100%|██████████| 788544/788544 [00:04<00:00, 167364.53it/s]


In [15]:
test2 = make_dataset2(test_err,3,14999,30000).move_errtype()
test3 = make_dataset2(test_err,7,14999,30000).move_errtype()
test4 = make_dataset2(qual2,7,14999,30000).make_qual_data()

100%|██████████| 725208/725208 [00:04<00:00, 167566.30it/s]


### 2-2-3. make_dataset3

In [160]:
class mk_dataset3(): # train_err_data 관련 파생변수 생성 
    
    def __init__(self, df, types, num_user, num_errtype, first):
        self.df = df
        self.types = types
        self.num_user = num_user
        self.num_errtype = num_errtype
        self.first = first
        self.num_model_change =2
        
    def run(self):
        vrb1 = self.errtype_cnt() # 에러타입마다 카운트
        vrb2 =self.model_change() # 모델 변화
        vrb3 =self.fw_change() # 펌웨어 버전 변화
        m, std =self.errtype_sta() # 총 에러타입 통계수치
        mhour, stdhour, maxhour =self.errtype_of_hour() # 시간별 에러타입 통계수치
        mday, stdday, maxday =self.errtype_of_day() # 일별 에러타입 통계수치
        vrb4 =self.fwver_flows() # 펌웨어버전 연관성
        vrb5 =self.get_time_term() # 시간별 에러발생 상황
        vrb6 =self.connetion_err() # connetion_err
        vrb7 =self.fwver_time() # 펌웨어 버전 변경까지의 시간
        vrb8=self.fw_err_cnt() # 주요 펌웨어 버전 카운트
        print(vrb1, vrb2, vrb3, m, std, mhour, stdhour, maxhour, mday, stdday, maxday, vrb4, vrb5, vrb6, vrb7, vrb8)
        
        
        return [vrb3,vrb2, vrb1, m, std ,mhour, stdhour, maxhour,mday, maxday, stdday, vrb4 , vrb5 ,vrb6  ,vrb7,vrb8]
        
    def errtype_cnt (self) : 
        errtypes = np.zeros((self.num_user,self.num_errtype))

        for idx, value in tqdm(self.df[['user_id','errtype']].values):
            errtypes[idx-self.first,value-1] +=1
#         print(errtypes.shape)
        return errtypes


    def model_change(self): 
        df = self.df
        first = self.first
        
        v=df[['user_id','model_nm']]
        getdf =~(v == v.shift(1))
        logical =(getdf.user_id.apply(int) + getdf.model_nm.apply(int)) > 0
        df_model_counts =v[logical]


        def get_model_change_id(num):
            df_mc = df_model_counts.user_id.value_counts()
            df_mc_user=list(df_mc.loc[df_mc ==num].to_frame().index)
            sort_mc_user = df.loc[df.user_id.isin(df_mc_user)].drop_duplicates(
                ['user_id','model_nm'],keep='last').drop_duplicates('user_id', keep='first').sort_values("time").user_id
            return sort_mc_user.values

        one_m = get_model_change_id(1)
        two_m = get_model_change_id(2)
        thr_m = get_model_change_id(3)


        model_n = np.zeros((self.num_user,self.num_model_change))
        df['model_f'] =df['model_nm'].str[-1].astype('int')
        one_df = df.loc[df.user_id.isin(one_m)][['user_id','model_f']].drop_duplicates().values
        two_df =df.loc[df.user_id.isin(two_m)][['user_id','model_f']].drop_duplicates().reset_index(drop=True)
        two_df['tf'] = two_df.index%2
        two_df= two_df.pivot(index='user_id',columns='tf').reset_index().values
        thr_df = df.loc[df.user_id.isin(thr_m)][['user_id','model_f']].drop_duplicates().values

        for inx, value in tqdm(one_df):
            model_n[inx-self.first,0]  +=value
        for inx, value1,value2 in tqdm(two_df):
            model_n[inx-self.first,0]  +=value1
            model_n[inx-self.first,1]  +=value2

        for inx, value in tqdm(thr_df):
            model_n[inx-self.first,0]  +=value
        
#         print(model_n.shape)
        return model_n

    def fw_change(self): 
        df= self.df
        
        fwver_total_dic ={}
        for v in range(len(fwver_total)):
            fwver_total_dic[sorted(list(fwver_total))[v]] = v+1


        df['ver_num'] = df['fwver'].apply(lambda x : fwver_total_dic[x])
        fwver_np = np.zeros((self.num_user,5))

        v=df[['user_id','ver_num']]
        getdf =~(v == v.shift(1))
        logical =(getdf.user_id.apply(int) + getdf.ver_num.apply(int)) > 0
        fwver_num=v[logical]
        
        fwver_num = fwver_num.reset_index(drop=True)
        count =np.zeros(len(fwver_num),dtype=int)

        for v in range(1,len(fwver_num)):
            if fwver_num.user_id.values[v-1] ==fwver_num.user_id.values[v]:
                count[v] = count[v-1] +1


        fwver_num['count'] =count
        fw_v = fwver_num.loc[fwver_num['count'].isin([0,1,2,3,4])].pivot(
            index='user_id',columns='count').reset_index().fillna(0).values
        fw_v =fw_v.astype('int64')

        for inx, v1,v2,v3,v4,v5 in tqdm(fw_v):
            fwver_np[inx-self.first,0] =v1
            fwver_np[inx-self.first,1] =v2
            fwver_np[inx-self.first,2] =v3
            fwver_np[inx-self.first,3] =v4
            fwver_np[inx-self.first,4] =v5
#         print(fwver_np.shape)
        return fwver_np

    def errtype_sta(self):
        df = self.df
        
        df['week'] =self.df.time.dt.isocalendar().week

        df = df.loc[(df.time >=pd.to_datetime('2020-11-01 00:00:00')) & (df.time<= pd.to_datetime('2020-11-30 23:59:59'))]
        datas = df[['user_id','errtype','week']]
        df_=datas[['user_id','week','errtype']].value_counts().to_frame().reset_index()
        df_ =df_.sort_values(['user_id','week']).rename(columns = {0:'counts'}).reset_index(drop=True)

        df1 =df_.loc[df_.week ==44][['user_id','errtype','counts']].values
        df2 =df_.loc[df_.week ==45][['user_id','errtype','counts']].values
        df3 =df_.loc[df_.week ==46][['user_id','errtype','counts']].values
        df4 =df_.loc[df_.week ==47][['user_id','errtype','counts']].values
        df5 =df_.loc[df_.week ==48][['user_id','errtype','counts']].values

        day_data = np.zeros((self.num_user,42,5))
        for i, dfa in enumerate([df1,df2,df3,df4,df5]):
            for inx , val1 ,val2 in tqdm(dfa):
                day_data[:,:,i][inx-self.first,val1-1] = val2

        m=day_data.mean(axis=2)
        std=day_data.std(axis=2)
        
#         print(m.shape)
#         print(std.shape)
        return m, std
        
    def errtype_of_hour (self):
        df = self.df
        df['hour'] =df.time.dt.hour


        df = df.loc[(df.time >=pd.to_datetime('2020-11-01 00:00:00')) & (df.time<= pd.to_datetime('2020-11-30 23:59:59'))]
        datas = df[['user_id','errtype','hour']]
        df_=datas[['user_id','hour','errtype']].value_counts().to_frame().reset_index()
        df_ =df_.sort_values(['user_id','hour']).rename(columns = {0:'counts'}).reset_index(drop=True)


        day_data = np.zeros((self.num_user,42,24))
        for i in range(24):
            dfa = df_.loc[df_['hour']==i][['user_id','errtype','counts']].values
            for inx , val1 ,val2 in tqdm(dfa):
                day_data[:,:,i][inx-self.first,val1-1] = val2

        mhour=day_data.mean(axis=2)
        stdhour=day_data.std(axis=2)       
        maxhour=day_data.max(axis=2)
        
#         print(mhour.shape)
#         print(stdhour.shape)
#         print(maxhour.shape)
        return mhour, stdhour, maxhour
 
    def errtype_of_day (self):
        df = self.df
        
        df['day'] =df.time.dt.day


        df = df.loc[(df.time >=pd.to_datetime('2020-11-01 00:00:00')) & (df.time<= pd.to_datetime('2020-11-30 23:59:59'))]
        datas = df[['user_id','errtype','day']]
        df_=datas[['user_id','day','errtype']].value_counts().to_frame().reset_index()
        df_ =df_.sort_values(['user_id','day']).rename(columns = {0:'counts'}).reset_index(drop=True)

        day_data = np.zeros((self.num_user,42,30))
        for i in range(30):
            dfa = df_.loc[df_['day']==(i+1)][['user_id','errtype','counts']].values
            for inx , val1 ,val2 in tqdm(dfa):
                day_data[:,:,i][inx-self.first,val1-1] = val2

        mday=day_data.mean(axis=2)
        stdday=day_data.std(axis=2)       
        maxday=day_data.max(axis=2)
        
#         print(mday.shape)
#         print(stdday.shape)
#         print(maxday.shape)
        return mday, stdday, maxday
    
    
    def fwver_flows(self):
        target_df = self.df
        first_num = self.first
        count_num =self.num_user

        dp = target_df[['user_id','model_nm','fwver']]
        unique_data =target_df[(dp !=dp.shift(1)).sum(axis=1)>0]

        dp2 = target_df[['user_id','model_nm']]
        unique_data2 =target_df[(dp2 !=dp2.shift(1)).sum(axis=1)>0]

        fwver_total_dic ={}
        for v in range(len(fwver_total)):
            fwver_total_dic[sorted(list(fwver_total))[v]] = v+1


        fwver = np.zeros((count_num,24))
        for idx in tqdm(unique_data.user_id.unique()):
            df_md =unique_data2.loc[unique_data2.user_id==idx].model_nm.values
            df_fw = unique_data.loc[unique_data.user_id==idx].fwver.values

            for md in range(len(df_md)):
                fwver[idx-first_num,md] = int(df_md[md][-1])+1

            for l in range(3,len(df_fw)+3):
                fwver[idx-first_num,l] =fwver_total_dic[df_fw[l-3]]

        fw_df = pd.DataFrame(fwver).reset_index().rename(columns={'index':'user_id'})

        fwver_total_dic_rev = {v: k for k, v in fwver_total_dic.items()}
        fwver_total_dic_rev2 = fwver_total_dic_rev.copy()
        fwver_total_dic_rev[0] =0
        fwver_total_dic_rev2[0] = '04.22.1750'


        fw_df[3] =fw_df[3].apply(lambda x : fwver_total_dic_rev2[x])
        for _ in range(4,8):
            fw_df[_] =fw_df[_].apply(lambda x : fwver_total_dic_rev[x])


        fw_df = fw_df.rename(columns={0:'md1',1:'md2',2:'md3',3:'fw1',4:'fw2',5:'fw3',6:'fw4',7:'fw5'})
        fw_df['user_id'] =fw_df['user_id']+10000

        pre_df=fw_df.iloc[:,:9]

        md_flow = {str(x.astype("int")):(i+1) for i,x in enumerate(
            pre_df[['md1','md2','md3']].drop_duplicates().reset_index(drop=True).values)}
        
        fw_flow = {str(x):(i+1) for i,x in enumerate(
            pre_df[['fw1','fw2','fw3','fw4','fw5']].drop_duplicates().reset_index(drop=True).values)}


        def fw_change_counter(x):
            fwlst = []
            for v in ['fw1','fw2','fw3','fw4','fw5']:
                if x[v] ==0:
                    pass
                else:
                    fwlst +=[x[v]]
            if len(fwlst) ==len(list(set(fwlst))):
                return 0
            else:
                return 1

        def mean_str_fw_dum(x):
            fwlst = []
            for v in ['fw1','fw2','fw3','fw4','fw5']:
                if x[v] ==0:
                    pass
                else:
                    fwlst +=[int(x[v].replace('.',""))]
            return np.array(fwlst).mean()

        def std_str_fw_dum(x):
            fwlst = []
            for v in ['fw1','fw2','fw3','fw4','fw5']:
                if x[v] ==0:
                    pass
                else:
                    fwlst +=[int(x[v].replace('.',""))]
            return np.array(fwlst).std()

        pre_df=fw_df.iloc[:,:9]
        pre_df['md_counts'] = pre_df[['md1','md2','md3']].astype('bool').sum(axis=1)
        pre_df['fw_counts'] = pre_df[['fw1','fw2','fw3','fw4','fw5']].astype('bool').sum(axis=1)

        pre_df['fw_change'] = pre_df.apply(fw_change_counter,axis=1)
        pre_df['fw_flows'] = pre_df.apply(lambda x : fw_flow[str(x[['fw1','fw2','fw3','fw4','fw5']].values)],axis=1)
        pre_df['md_flows'] = pre_df.apply(lambda x :md_flow[str(x[['md1','md2','md3']].values.astype("int"))],axis=1)
        pre_df['fw_mean'] = pre_df.apply(mean_str_fw_dum,axis=1)
        pre_df['fw_std'] = pre_df.apply(std_str_fw_dum,axis=1)


        fw_model_flow =pre_df.iloc[:,9:].values
        
#         print(fw_model_flow.shape)
        return fw_model_flow
    
    def get_time_term(self):
        df = self.df
        first_num = self.first
        count_num = self.num_user

        time_term = np.zeros((count_num,4))
        tmp =df[['user_id','time']].drop_duplicates()

        for v in tqdm(tmp.user_id.unique()):
            test =tmp.loc[tmp.user_id ==v].time
            if len(test) <=2:
                time_term[v-first_num,0] = 0
                time_term[v-first_num,1] = 0
                time_term[v-first_num,2] = test.values[-1]-test.values[0]
                time_term[v-first_num,3] = len(test)
            else:
                time_term[v-first_num,0] = (test -test.shift(1)).max().total_seconds()
                time_term[v-first_num,1] = (test -test.shift(1)).min().total_seconds()
                time_term[v-first_num,2] = test.values[-1]-test.values[0]  
                time_term[v-first_num,3] = len(test)

        dft = pd.DataFrame(time_term).copy()

        dft[0] =dft[0]/3600
        dft[2] =dft[2]/3600/24/10e8
        dft[2] =np.where(dft[2].values==0,1,dft[2].values)
        dft[5] =dft[0]/dft[3]
        dft[5] = dft[0]/dft[3]*3600
        dft[6] = dft[0]/24/dft[2]
        
#         print(dft.fillna(0).values.shape)
        return dft.fillna(0).values
        
    def connetion_err(self):
        first_num = self.first
        count_num = self.num_user
        
        err_df =self.df[['user_id','time','errcode']].dropna(axis=0)

        df_con =err_df.loc[err_df.errcode.str.contains("connection")]
        df_con['check_time'] = df_con.time.dt.date
        df_con['check_hour'] =df_con.time.dt.hour

        def connetion_df(con_df):
            day_con_err =con_df[['user_id','check_time']].value_counts().groupby("user_id").max()
            hour_con_err=con_df.groupby(['user_id','check_time','check_hour']).size().groupby("user_id").max()
            tenmin_con_err =con_df.set_index('time').groupby(['user_id','errcode']).resample("10min").size().groupby("user_id").max()
            con_trans =pd.concat([day_con_err,hour_con_err,tenmin_con_err],axis=1)
            return con_trans

        total_conn_err=[connetion_df(df_con)]
        for errs in tqdm(['connection timeout', 'connection fail to establish','connectionterminated by local host',
                          'connection fail for LMP response timout','L2CAP connection cancelled']):
            
            con_esta = df_con.loc[df_con.errcode.str.contains(errs)]
            total_conn_err.append(connetion_df(con_esta))

        base_df =pd.DataFrame(range(first_num,first_num+count_num)).rename(columns={0:'user_id'}).set_index('user_id')

        connetion_err_pre = pd.concat(total_conn_err+[base_df],axis=1).fillna(0).values
        
#         print(connection_err_pre.shape)
        return connetion_err_pre
    
    def fwver_time(self):
        
        fwver_total_dic ={}
        for v in range(len(fwver_total)):
            fwver_total_dic[sorted(list(fwver_total))[v]] = v+1
            
        tsed = self.df.dropna(axis=0).reset_index(drop=True)[['user_id','time','fwver']]
        dfw = tsed[['user_id','fwver']]
        fw_d =dfw.loc[(dfw !=dfw.shift(1)).sum(axis=1)>0]

        main_fw_ar = np.zeros((self.num_user,6))
        for i,tgid in enumerate(tqdm(range(self.first,self.first+self.num_user))):

            tgdf =fw_d.loc[fw_d.user_id ==tgid].iloc[1:,:]
            tgidtotal = tsed.loc[tsed.user_id ==tgid]
            
            try:
                data =tgidtotal.loc[sorted([tgidtotal.index[0]] + [x-1 for x in tgdf.index]+[x for x in tgdf.index] + [tgidtotal.index[-1]] )]
                t1 =data.time
                time_delta = (t1-t1.shift(1)).dt.total_seconds()

                main_fwver =data.loc[time_delta.loc[time_delta==time_delta.max()].index].fwver.values[0]
                main_fw_ar[i,0] = fwver_total_dic[main_fwver]
                main_fw_ar[i,1] =(time_delta[1::2].values).max().astype('float')/(time_delta.values[1:]).sum().astype('float')  #target fw workingtime / total
                if len(time_delta) ==1:
                    main_fw_ar[i,2] =0  
                    main_fw_ar[i,3] =0  
                    main_fw_ar[i,4] =0  
                    main_fw_ar[i,5] =0  
                else:
                    main_fw_ar[i,2] =time_delta[::2].min()/3600 
                    main_fw_ar[i,3] =time_delta[::2].std()/3600
                    main_fw_ar[i,4] =time_delta[1::2].values.astype('float').std()/3600 
                    main_fw_ar[i,5] =(time_delta[1::2].values.astype('float')/3600).var()  
            except:
                main_fw_ar[i,0] =0
                main_fw_ar[i,1] =0
                main_fw_ar[i,2] =0
                main_fw_ar[i,3] =0
                main_fw_ar[i,4] =0
                main_fw_ar[i,5] =0
                
#         print(main_fw_ar.shape)
        return main_fw_ar
    
    def fw_err_cnt(self):
        fwver_total_dic ={}
        for v in range(len(fwver_total)):
            fwver_total_dic[sorted(list(fwver_total))[v]] = v+1
            
        tred = self.df.dropna(axis=0).reset_index(drop=True)[['user_id','time','fwver']]
        tred_all = self.df.dropna(axis=0).reset_index(drop=True)
        
        tsed = self.df.dropna(axis=0).reset_index(drop=True)[['user_id','time','fwver']]
        dfw = tsed[['user_id','fwver']]
        fw_d =dfw.loc[(dfw !=dfw.shift(1)).sum(axis=1)>0]

        main_fw_err_counts = np.zeros((self.num_user,84))
        for i,tgid in enumerate(tqdm(range(self.first,self.first+self.num_user))):

            tgdf =fw_d.loc[fw_d.user_id ==tgid].iloc[1:,:]
            tgidtotal = tred_all.loc[tred_all.user_id ==tgid]

            
            try:
                data =tgidtotal.loc[sorted([tgidtotal.index[0]] + [x-1 for x in tgdf.index]+[x for x in tgdf.index] + [tgidtotal.index[-1]] )]
                t1 =data.time
                time_delta = (t1-t1.shift(1)).dt.total_seconds()

                main_fwver =data.loc[time_delta.loc[time_delta==time_delta.max()].index].fwver.values[0]


                main_date = tgidtotal.loc[tgidtotal.fwver ==main_fwver][['time','errtype']]
                main_date['date'] =main_date.time.dt.date
                main_time_del  =(main_date.time.dt.date.values[-1] -main_date.time.dt.date.values[0]).days
                date_first= main_date.time.dt.date.values[0]
                err_count_main = np.zeros((42,main_time_del+1))
                for n in  range(main_time_del+1):
                    lcdf = main_date.loc[main_date.date ==date_first]
                    for errtype in lcdf.errtype.values:
                        err_count_main[errtype-1,n] +=1
                    date_first += dt.timedelta(days=1)

                main_fw_err_counts[i,:42] =     err_count_main.mean(axis=1)
                main_fw_err_counts[i,42:] =     err_count_main.std(axis=1)

            except:
                main_fw_err_counts[i,:] = 0
        
#         print(main_fw_err_counts.shape)
        return main_fw_err_counts

In [161]:
train6 = mk_dataset3(train_err2,'train',15000,42,10000).run()
train6 = pd.DataFrame(np.concatenate(tuple(train6),axis=1))

100%|██████████| 15000/15000 [07:44<00:00, 32.29it/s]


[[  0.   0.   8. ...   0.   0.   0.]
 [  0.   0.   0. ... 113.  56.   1.]
 [  0.   0.   2. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ...  58.   8.   5.]
 [  0.   0.   0. ...   6.   0.   0.]
 [  0.   0.   4. ...   0.   0.   0.]] [[3. 0.]
 [2. 0.]
 [3. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [3. 0.]] [[39.  0.  0.  0.  0.]
 [25. 26.  0.  0.  0.]
 [39.  0.  0.  0.  0.]
 ...
 [18. 20.  0.  0.  0.]
 [18. 20.  0.  0.  0.]
 [39.  0.  0.  0.  0.]] [[ 0.   0.   1.6 ...  0.   0.   0. ]
 [ 0.   0.   0.  ... 22.6 10.8  0.2]
 [ 0.   0.   0.4 ...  0.   0.   0. ]
 ...
 [ 0.   0.   0.  ... 11.4  1.6  1. ]
 [ 0.   0.   0.  ...  0.8  0.   0. ]
 [ 0.   0.   0.8 ...  0.   0.   0. ]] [[ 0.          0.          1.62480768 ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ... 17.15342531  7.98498591
   0.4       ]
 [ 0.          0.          0.8        ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ... 12.41933976  3.2
   1.54919334]
 [ 0.          0.          0.     

In [18]:
test6 = mk_dataset3(test_err2,'test',14999,42,30000).run()
test6 = pd.DataFrame(np.concatenate(tuple(test6),axis=1))

100%|██████████| 14999/14999 [07:35<00:00, 32.90it/s]


---

## Developed Code (2)
* Developed Code(1)은 Class내 함수 최적화 및 변경입니다.

###### 1. Quality값이 -1인 Data로 만든 Feature 추가

In [19]:
train_qual = pd.read_csv("./preprocessing/train_quality_data_time.csv")
test_qual = pd.read_csv("./preprocessing/test_quality_data_time.csv")

In [20]:
def move_quality_minus(data, number, min_id, max_id):
    tqdm.pandas()
    data2 = data.copy()
    ap_data_tr = pd.DataFrame({"user_id" : np.arange(min_id, max_id)})

    # 양수값 -1로 대체
    for i in data2.columns[3:]:
        idx = data2[data2[i] != -1].index
        data2.loc[idx, i] = 0

    # -----------------------------------------------------------------------------------------------------
    # 이동평균의 평균
    mean_data = data2.groupby(['user_id','time'])['quality_' + str(number)].rolling(window=12).mean().\
    to_frame().reset_index().dropna()
    del mean_data['level_2']
    mean_data = mean_data.groupby('user_id')['quality_' + str(number)].mean().to_frame().reset_index()
    mean_data = pd.merge(ap_data_tr, mean_data, how='left', on = 'user_id').fillna(0)
    mean_data = mean_data.iloc[:,1]


    # 이동최대값의 최대값
    max_data = data2.groupby(['user_id','time'])['quality_' + str(number)].rolling(window=12).max().\
    to_frame().reset_index().dropna()
    del max_data['level_2']
    max_data = max_data.groupby('user_id')['quality_' + str(number)].max().to_frame().reset_index()
    max_data = pd.merge(ap_data_tr, max_data, how='left', on = 'user_id').fillna(0)
    max_data = max_data.iloc[:,1]


    # 이동최소값의 최소값
    min_data = data2.groupby(['user_id','time'])['quality_' + str(number)].rolling(window=12).min().\
    to_frame().reset_index().dropna()
    del min_data['level_2']
    min_data = min_data.groupby('user_id')['quality_' + str(number)].min().to_frame().reset_index()
    min_data = pd.merge(ap_data_tr, min_data, how='left', on = 'user_id').fillna(0)
    min_data = min_data.iloc[:,1]


    # 이동표준편차의 평균
    std_data = data2.groupby(['user_id','time'])['quality_' + str(number)].rolling(window=12).std().\
    to_frame().reset_index().dropna()
    del std_data['level_2']
    std_data = std_data.groupby('user_id')['quality_' + str(number)].mean().to_frame().reset_index()
    std_data = pd.merge(ap_data_tr, std_data, how='left', on = 'user_id').fillna(0)
    std_data = std_data.iloc[:,1]


    # 이동평균의 최대값
    meanmax_data = data2.groupby(['user_id','time'])['quality_' + str(number)].rolling(window=12).mean().\
    to_frame().reset_index().dropna()
    del meanmax_data['level_2']
    meanmax_data = meanmax_data.groupby('user_id')['quality_' + str(number)].max().to_frame().reset_index()
    meanmax_data = pd.merge(ap_data_tr, meanmax_data, how='left', on = 'user_id').fillna(0)
    meanmax_data = meanmax_data.iloc[:,1]


    # 이동최대값의 평균
    maxmean_data = data2.groupby(['user_id','time'])['quality_' + str(number)].rolling(window=12).max().\
    to_frame().reset_index().dropna()
    del maxmean_data['level_2']
    maxmean_data = maxmean_data.groupby('user_id')['quality_' + str(number)].mean().to_frame().reset_index()
    maxmean_data = pd.merge(ap_data_tr, maxmean_data, how='left', on = 'user_id').fillna(0)
    maxmean_data = maxmean_data.iloc[:,1]


    # merge
    moving_result = pd.concat([mean_data, max_data, min_data, std_data, meanmax_data, maxmean_data], axis=1)

    moving_result.columns = ['Quality_'+str(number)+'_mean_d', 'Quality_'+str(number)+'_max_d',
                             'Quality_'+str(number)+'_min_d', 'Quality_'+str(number)+'_std_d',
                             'Quality_'+str(number)+'_meanmax_d', 'Quality_'+str(number)+'_maxmean_d']

    return moving_result

In [21]:
res0 = move_quality_minus(train_qual, 0, 10000, 25000)
res1 = move_quality_minus(train_qual, 1, 10000, 25000)
res2 = move_quality_minus(train_qual, 2, 10000, 25000)
res3 = move_quality_minus(train_qual, 3, 10000, 25000)
res4 = move_quality_minus(train_qual, 4, 10000, 25000)
res5 = move_quality_minus(train_qual, 5, 10000, 25000)
res6 = move_quality_minus(train_qual, 6, 10000, 25000)
res7 = move_quality_minus(train_qual, 7, 10000, 25000)
res8 = move_quality_minus(train_qual, 8, 10000, 25000)
res9 = move_quality_minus(train_qual, 9, 10000, 25000)
res10 = move_quality_minus(train_qual, 10, 10000, 25000)
res11 = move_quality_minus(train_qual, 11, 10000, 25000)
res12 = move_quality_minus(train_qual, 12, 10000, 25000)

res_train = pd.concat([res0, res1, res2, res3, res4, res5, res6, res7, res8, res9, res10, res11, res12], axis=1)

In [22]:
res0_t = move_quality_minus(test_qual, 0, 30000, 44999)
res1_t = move_quality_minus(test_qual, 1, 30000, 44999)
res2_t = move_quality_minus(test_qual, 2, 30000, 44999)
res3_t = move_quality_minus(test_qual, 3, 30000, 44999)
res4_t = move_quality_minus(test_qual, 4, 30000, 44999)
res5_t = move_quality_minus(test_qual, 5, 30000, 44999)
res6_t = move_quality_minus(test_qual, 6, 30000, 44999)
res7_t = move_quality_minus(test_qual, 7, 30000, 44999)
res8_t = move_quality_minus(test_qual, 8, 30000, 44999)
res9_t = move_quality_minus(test_qual, 9, 30000, 44999)
res10_t = move_quality_minus(test_qual, 10, 30000, 44999)
res11_t = move_quality_minus(test_qual, 11, 30000, 44999)
res12_t = move_quality_minus(test_qual, 12, 30000, 44999)

res_test = pd.concat([res0_t, res1_t, res2_t, res3_t, res4_t, res5_t, res6_t,
                      res7_t, res8_t, res9_t, res10_t, res11_t, res12_t], axis=1)

---

## 2-4. 최종 데이터 병합

In [23]:
def select_concat(df1,df2,df3,df4,df5) :# 변수 선택 및 데이터프레임 병합
    select1 = df5.iloc[ : , 0:8]
    select2 = df5.iloc[ : , 50:92]
    select3 = df5.iloc[ : , 134:]

    final_df = pd.concat([df1, df2, df3, df4,
                         select1, select2, select3], axis=1)
    
    for i in final_df.columns:
        str_i = str(i)
        if 'diff' in str_i :
            final_df.drop(i, axis='columns', inplace=True)
    return final_df

In [24]:
train_final = select_concat(train1, train2, train3, train4 ,train6)
test_final = select_concat(test1, test2, test3,test4, test6)

In [25]:
train_final = pd.concat([train_final, res_train], axis=1)
test_final = pd.concat([test_final, res_test], axis=1)

In [26]:
train_final.to_csv(SAVE_PATH + "./trainf_developed.csv", index = False)
test_final.to_csv(SAVE_PATH + "./testf_developed.csv", index = False)

---

# 모델 훈련 및 예측

## 3-1. 모델학습

In [27]:
# train_final = pd.read_csv(SAVE_PATH + "./trainf_developed.csv")
# test_final = pd.read_csv(SAVE_PATH + "./testf_developed.csv")

# 결측치가 많은 칼럼 처리
del train_final[419]
del train_final[418]
train_final[417] = train_final[417].fillna(1.000000)

del test_final[419]
del test_final[418]
test_final[417] = test_final[417].fillna(1.000000)
test_final = test_final.fillna(0)

In [28]:
# y값 생성
label = np.zeros(15000)
label[train_problem.user_id.unique()-10000] = 1 

In [29]:
train_x = train_final.copy()
train_y = label.astype(int)

# 카테고리 변수 지정
train_x['ver'] = train_x['ver'].astype('category')
train_x['ver']= train_x['ver'].cat.codes

test_final['ver'] = test_final['ver'].astype('category')
test_final['ver']= test_final['ver'].cat.codes

(train_x.shape, train_y.shape)

((15000, 1066), (15000,))

In [30]:
# 카테고리 변수 추가 지정
train_x.rename(columns = {0 : 'fw1',
                          1 : 'fw2',
                          2 : 'fw3',
                          3 : 'fw4',
                          4 : 'fw5',
                          5 : 'mn1',
                          6 : 'mn2',
                          7 : 'mn3'}, inplace=True)

test_final.rename(columns = {0 : 'fw1',
                          1 : 'fw2',
                          2 : 'fw3',
                          3 : 'fw4',
                          4 : 'fw5',
                          5 : 'mn1',
                          6 : 'mn2',
                          7 : 'mn3'}, inplace=True)

In [31]:
for i in list(train_x.columns)[100:]:
    if i in ['fw1', 'fw2', 'fw3', 'fw4', 'fw5', 'mn1', 'mn2', 'mn3']:
        train_x[i] = train_x[i].astype(str)

In [32]:
for i in list(test_final.columns)[100:]:
    if i in ['fw1', 'fw2', 'fw3', 'fw4', 'fw5', 'mn1', 'mn2', 'mn3']:
        test_final[i] = test_final[i].astype(str)

In [33]:
train_x.head()

,err_1_cnt,err_2_cnt,err_3_cnt,err_4_cnt,err_5_cnt,err_6_cnt,err_7_cnt,err_8_cnt,err_9_cnt,err_10_cnt,...,Quality_11_min_d,Quality_11_std_d,Quality_11_meanmax_d,Quality_11_maxmean_d,Quality_12_mean_d,Quality_12_max_d,Quality_12_min_d,Quality_12_std_d,Quality_12_meanmax_d,Quality_12_maxmean_d
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,...,-1.0,0.048656,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,...,-1.0,0.194625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
# 파라미터가 다른 두 캣부스트 모델을 동시에 학습후 블렌딩

def dual_blending (train_x, train_y):
    
    models1    = []
    auc_scores1   = []
    models2    = []
    auc_scores2   = []

    threshold = 0.5

    # 5 Kfold cross validation
    k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for train_idx, val_idx in k_fold.split(train_x,train_y):

        X = train_x.iloc[train_idx,:]
        y = train_y[train_idx]
        valid_x = train_x.iloc[val_idx,:]
        valid_y = train_y[val_idx]

        # 첫번째 모델
        model1 = CatBoostRegressor(iterations=1000, l2_leaf_reg = 0.01, random_seed = 1422)
        model1.fit(X,y, cat_features = ['ver', 'fw1', 'fw2', 'fw3', 'fw4', 'fw5', 'mn1', 'mn2', 'mn3'])


        valid_prob1 = model1.predict(valid_x)
        valid_pred1 = np.where(valid_prob1 > threshold, 1, 0)
        auc_score1 = roc_auc_score(   valid_y, valid_prob1)


        models1.append(model1)
        auc_scores1.append(auc_score1)

        print('==========================================================')

        # 두번째 모델
        model2 = CatBoostRegressor(iterations=1000, l2_leaf_reg = 0.003, random_seed = 1422)
        model2.fit(X,y, cat_features = ['ver', 'fw1', 'fw2', 'fw3', 'fw4', 'fw5', 'mn1', 'mn2', 'mn3'])


        valid_prob2 = model2.predict(valid_x)
        valid_pred2 = np.where(valid_prob2 > threshold, 1, 0)
        auc_score2 = roc_auc_score(valid_y, valid_prob2)


        models2.append(model2)
        auc_scores2.append(auc_score2)

        print('==========================================================')
        
    return models1, models2, auc_scores1, auc_scores2

In [35]:
group1, group2, scores1, scores2 = dual_blending(train_x, train_y)

0:	learn: 0.4674442	total: 145ms	remaining: 2m 24s
1:	learn: 0.4636843	total: 191ms	remaining: 1m 35s
2:	learn: 0.4602679	total: 265ms	remaining: 1m 28s
3:	learn: 0.4568581	total: 301ms	remaining: 1m 14s
4:	learn: 0.4536236	total: 378ms	remaining: 1m 15s
5:	learn: 0.4505679	total: 421ms	remaining: 1m 9s
6:	learn: 0.4478717	total: 495ms	remaining: 1m 10s
7:	learn: 0.4450149	total: 535ms	remaining: 1m 6s
8:	learn: 0.4424391	total: 615ms	remaining: 1m 7s
9:	learn: 0.4399396	total: 653ms	remaining: 1m 4s
10:	learn: 0.4375152	total: 721ms	remaining: 1m 4s
11:	learn: 0.4350655	total: 753ms	remaining: 1m 2s
12:	learn: 0.4328979	total: 824ms	remaining: 1m 2s
13:	learn: 0.4307990	total: 858ms	remaining: 1m
14:	learn: 0.4288130	total: 930ms	remaining: 1m 1s
15:	learn: 0.4269830	total: 961ms	remaining: 59.1s
16:	learn: 0.4251142	total: 1.03s	remaining: 59.6s
17:	learn: 0.4234531	total: 1.06s	remaining: 58s
18:	learn: 0.4217010	total: 1.13s	remaining: 58.5s
19:	learn: 0.4201202	total: 1.17s	remain

162:	learn: 0.3723752	total: 8.71s	remaining: 44.7s
163:	learn: 0.3722954	total: 8.74s	remaining: 44.5s
164:	learn: 0.3721818	total: 8.8s	remaining: 44.5s
165:	learn: 0.3720717	total: 8.84s	remaining: 44.4s
166:	learn: 0.3720090	total: 8.91s	remaining: 44.4s
167:	learn: 0.3718968	total: 8.94s	remaining: 44.3s
168:	learn: 0.3717498	total: 9.01s	remaining: 44.3s
169:	learn: 0.3716461	total: 9.04s	remaining: 44.1s
170:	learn: 0.3715832	total: 9.11s	remaining: 44.2s
171:	learn: 0.3715137	total: 9.14s	remaining: 44s
172:	learn: 0.3714555	total: 9.21s	remaining: 44s
173:	learn: 0.3714055	total: 9.25s	remaining: 43.9s
174:	learn: 0.3712758	total: 9.32s	remaining: 43.9s
175:	learn: 0.3712047	total: 9.35s	remaining: 43.8s
176:	learn: 0.3710973	total: 9.42s	remaining: 43.8s
177:	learn: 0.3710224	total: 9.46s	remaining: 43.7s
178:	learn: 0.3709860	total: 9.53s	remaining: 43.7s
179:	learn: 0.3708604	total: 9.56s	remaining: 43.5s
180:	learn: 0.3707978	total: 9.63s	remaining: 43.6s
181:	learn: 0.370

324:	learn: 0.3590430	total: 17.3s	remaining: 35.9s
325:	learn: 0.3589689	total: 17.3s	remaining: 35.8s
326:	learn: 0.3588704	total: 17.4s	remaining: 35.8s
327:	learn: 0.3588078	total: 17.4s	remaining: 35.7s
328:	learn: 0.3587343	total: 17.5s	remaining: 35.7s
329:	learn: 0.3586589	total: 17.5s	remaining: 35.6s
330:	learn: 0.3585435	total: 17.6s	remaining: 35.6s
331:	learn: 0.3584795	total: 17.6s	remaining: 35.5s
332:	learn: 0.3583212	total: 17.7s	remaining: 35.5s
333:	learn: 0.3582306	total: 17.8s	remaining: 35.4s
334:	learn: 0.3581315	total: 17.8s	remaining: 35.4s
335:	learn: 0.3580324	total: 17.9s	remaining: 35.3s
336:	learn: 0.3579007	total: 17.9s	remaining: 35.3s
337:	learn: 0.3578130	total: 18s	remaining: 35.2s
338:	learn: 0.3577353	total: 18s	remaining: 35.2s
339:	learn: 0.3576561	total: 18.1s	remaining: 35.1s
340:	learn: 0.3575114	total: 18.1s	remaining: 35.1s
341:	learn: 0.3573669	total: 18.2s	remaining: 35s
342:	learn: 0.3572544	total: 18.2s	remaining: 34.9s
343:	learn: 0.3571

484:	learn: 0.3428193	total: 25.7s	remaining: 27.3s
485:	learn: 0.3427174	total: 25.7s	remaining: 27.2s
486:	learn: 0.3426151	total: 25.8s	remaining: 27.2s
487:	learn: 0.3425360	total: 25.8s	remaining: 27.1s
488:	learn: 0.3424444	total: 25.9s	remaining: 27.1s
489:	learn: 0.3423342	total: 25.9s	remaining: 27s
490:	learn: 0.3422432	total: 26s	remaining: 27s
491:	learn: 0.3421143	total: 26s	remaining: 26.9s
492:	learn: 0.3420116	total: 26.1s	remaining: 26.9s
493:	learn: 0.3419429	total: 26.1s	remaining: 26.8s
494:	learn: 0.3418326	total: 26.2s	remaining: 26.7s
495:	learn: 0.3416877	total: 26.2s	remaining: 26.7s
496:	learn: 0.3415985	total: 26.3s	remaining: 26.6s
497:	learn: 0.3415047	total: 26.4s	remaining: 26.6s
498:	learn: 0.3413915	total: 26.4s	remaining: 26.5s
499:	learn: 0.3412640	total: 26.5s	remaining: 26.5s
500:	learn: 0.3412113	total: 26.5s	remaining: 26.4s
501:	learn: 0.3411220	total: 26.6s	remaining: 26.3s
502:	learn: 0.3410257	total: 26.6s	remaining: 26.3s
503:	learn: 0.340938

646:	learn: 0.3282241	total: 34.2s	remaining: 18.6s
647:	learn: 0.3281668	total: 34.2s	remaining: 18.6s
648:	learn: 0.3280622	total: 34.3s	remaining: 18.5s
649:	learn: 0.3279773	total: 34.3s	remaining: 18.5s
650:	learn: 0.3278866	total: 34.4s	remaining: 18.4s
651:	learn: 0.3277899	total: 34.4s	remaining: 18.4s
652:	learn: 0.3277104	total: 34.5s	remaining: 18.3s
653:	learn: 0.3276682	total: 34.5s	remaining: 18.3s
654:	learn: 0.3275921	total: 34.6s	remaining: 18.2s
655:	learn: 0.3275232	total: 34.6s	remaining: 18.2s
656:	learn: 0.3274367	total: 34.7s	remaining: 18.1s
657:	learn: 0.3273348	total: 34.8s	remaining: 18.1s
658:	learn: 0.3272848	total: 34.8s	remaining: 18s
659:	learn: 0.3271929	total: 34.9s	remaining: 18s
660:	learn: 0.3271202	total: 34.9s	remaining: 17.9s
661:	learn: 0.3270506	total: 35s	remaining: 17.9s
662:	learn: 0.3269553	total: 35s	remaining: 17.8s
663:	learn: 0.3269016	total: 35.1s	remaining: 17.8s
664:	learn: 0.3268268	total: 35.2s	remaining: 17.7s
665:	learn: 0.326727

...

310:	learn: 0.3593942	total: 18s	remaining: 39.8s
311:	learn: 0.3593089	total: 18s	remaining: 39.7s
312:	learn: 0.3592294	total: 18.1s	remaining: 39.6s
313:	learn: 0.3591700	total: 18.1s	remaining: 39.5s
314:	learn: 0.3590524	total: 18.2s	remaining: 39.5s
315:	learn: 0.3589535	total: 18.2s	remaining: 39.4s
316:	learn: 0.3588887	total: 18.3s	remaining: 39.4s
317:	learn: 0.3588300	total: 18.3s	remaining: 39.3s
318:	learn: 0.3587875	total: 18.4s	remaining: 39.2s
319:	learn: 0.3586654	total: 18.4s	remaining: 39.1s
320:	learn: 0.3585559	total: 18.5s	remaining: 39.1s
321:	learn: 0.3584495	total: 18.5s	remaining: 39s
322:	learn: 0.3583665	total: 18.6s	remaining: 39s
323:	learn: 0.3582603	total: 18.6s	remaining: 38.9s
324:	learn: 0.3581565	total: 18.7s	remaining: 38.8s
325:	learn: 0.3580756	total: 18.7s	remaining: 38.7s
326:	learn: 0.3579740	total: 18.8s	remaining: 38.7s
327:	learn: 0.3578850	total: 18.8s	remaining: 38.6s
328:	learn: 0.3577617	total: 18.9s	remaining: 38.6s
329:	learn: 0.357713

471:	learn: 0.3438137	total: 26.5s	remaining: 29.6s
472:	learn: 0.3437032	total: 26.5s	remaining: 29.6s
473:	learn: 0.3436184	total: 26.6s	remaining: 29.5s
474:	learn: 0.3435310	total: 26.6s	remaining: 29.5s
475:	learn: 0.3434103	total: 26.7s	remaining: 29.4s
476:	learn: 0.3433774	total: 26.8s	remaining: 29.3s
477:	learn: 0.3432869	total: 26.8s	remaining: 29.3s
478:	learn: 0.3431492	total: 26.9s	remaining: 29.2s
479:	learn: 0.3430512	total: 26.9s	remaining: 29.2s
480:	learn: 0.3429720	total: 27s	remaining: 29.1s
481:	learn: 0.3428159	total: 27s	remaining: 29s
482:	learn: 0.3427053	total: 27.1s	remaining: 29s
483:	learn: 0.3426122	total: 27.1s	remaining: 28.9s
484:	learn: 0.3425116	total: 27.2s	remaining: 28.9s
485:	learn: 0.3424382	total: 27.2s	remaining: 28.8s
486:	learn: 0.3424033	total: 27.3s	remaining: 28.8s
487:	learn: 0.3423422	total: 27.3s	remaining: 28.7s
488:	learn: 0.3422634	total: 27.4s	remaining: 28.6s
489:	learn: 0.3421754	total: 27.4s	remaining: 28.6s
490:	learn: 0.342073

632:	learn: 0.3298753	total: 34.9s	remaining: 20.2s
633:	learn: 0.3298366	total: 34.9s	remaining: 20.1s
634:	learn: 0.3297625	total: 35s	remaining: 20.1s
635:	learn: 0.3296748	total: 35s	remaining: 20s
636:	learn: 0.3296492	total: 35.1s	remaining: 20s
637:	learn: 0.3295659	total: 35.1s	remaining: 19.9s
638:	learn: 0.3294659	total: 35.2s	remaining: 19.9s
639:	learn: 0.3293612	total: 35.2s	remaining: 19.8s
640:	learn: 0.3293071	total: 35.3s	remaining: 19.8s
641:	learn: 0.3292149	total: 35.3s	remaining: 19.7s
642:	learn: 0.3291561	total: 35.4s	remaining: 19.6s
643:	learn: 0.3290693	total: 35.4s	remaining: 19.6s
644:	learn: 0.3289797	total: 35.5s	remaining: 19.5s
645:	learn: 0.3288762	total: 35.5s	remaining: 19.5s
646:	learn: 0.3287580	total: 35.6s	remaining: 19.4s
647:	learn: 0.3287061	total: 35.6s	remaining: 19.4s
648:	learn: 0.3285590	total: 35.7s	remaining: 19.3s
649:	learn: 0.3284827	total: 35.7s	remaining: 19.2s
650:	learn: 0.3284505	total: 35.8s	remaining: 19.2s
651:	learn: 0.328377

792:	learn: 0.3173139	total: 43.2s	remaining: 11.3s
793:	learn: 0.3172329	total: 43.3s	remaining: 11.2s
794:	learn: 0.3171494	total: 43.3s	remaining: 11.2s
795:	learn: 0.3170851	total: 43.4s	remaining: 11.1s
796:	learn: 0.3170004	total: 43.5s	remaining: 11.1s
797:	learn: 0.3169421	total: 43.5s	remaining: 11s
798:	learn: 0.3168867	total: 43.6s	remaining: 11s
799:	learn: 0.3168079	total: 43.6s	remaining: 10.9s
800:	learn: 0.3167707	total: 43.7s	remaining: 10.8s
801:	learn: 0.3166681	total: 43.7s	remaining: 10.8s
802:	learn: 0.3165667	total: 43.8s	remaining: 10.7s
803:	learn: 0.3164875	total: 43.8s	remaining: 10.7s
804:	learn: 0.3164440	total: 43.9s	remaining: 10.6s
805:	learn: 0.3163431	total: 43.9s	remaining: 10.6s
806:	learn: 0.3163069	total: 44s	remaining: 10.5s
807:	learn: 0.3162118	total: 44s	remaining: 10.5s
808:	learn: 0.3161150	total: 44.1s	remaining: 10.4s
809:	learn: 0.3160431	total: 44.1s	remaining: 10.3s
810:	learn: 0.3159867	total: 44.2s	remaining: 10.3s
811:	learn: 0.315904

952:	learn: 0.3053939	total: 51.7s	remaining: 2.55s
953:	learn: 0.3053601	total: 51.7s	remaining: 2.49s
954:	learn: 0.3052843	total: 51.8s	remaining: 2.44s
955:	learn: 0.3051971	total: 51.8s	remaining: 2.38s
956:	learn: 0.3051587	total: 51.9s	remaining: 2.33s
957:	learn: 0.3050772	total: 51.9s	remaining: 2.28s
958:	learn: 0.3050192	total: 52s	remaining: 2.22s
959:	learn: 0.3049917	total: 52s	remaining: 2.17s
960:	learn: 0.3049383	total: 52.1s	remaining: 2.11s
961:	learn: 0.3048723	total: 52.1s	remaining: 2.06s
962:	learn: 0.3047958	total: 52.2s	remaining: 2.01s
963:	learn: 0.3047637	total: 52.3s	remaining: 1.95s
964:	learn: 0.3047057	total: 52.3s	remaining: 1.9s
965:	learn: 0.3046725	total: 52.4s	remaining: 1.84s
966:	learn: 0.3046332	total: 52.5s	remaining: 1.79s
967:	learn: 0.3045600	total: 52.5s	remaining: 1.74s
968:	learn: 0.3044780	total: 52.6s	remaining: 1.68s
969:	learn: 0.3044041	total: 52.6s	remaining: 1.63s
970:	learn: 0.3043250	total: 52.7s	remaining: 1.57s
971:	learn: 0.304

## 예측 및 제출파일 생성

In [36]:
# 예측
pred_y_list1 = []
pred_y_list2 = []

for m1 in group1:
    pred_y1 = m1.predict(test_final)
    pred_y_list1.append(pred_y1.reshape(-1,1))
    
for m2 in group2:    
    pred_y2 = m2.predict(test_final)
    pred_y_list2.append(pred_y2.reshape(-1,1))
    
pred1 = np.mean(pred_y_list1, axis = 0)
pred2 = np.mean(pred_y_list2, axis = 0)
(pred1,pred2)

(array([[0.95018958],
        [0.38718603],
        [0.38562953],
        ...,
        [0.67275384],
        [0.95582961],
        [0.34287296]]),
 array([[0.92965741],
        [0.38412169],
        [0.39070718],
        ...,
        [0.68622281],
        [1.03006307],
        [0.37128154]]))

In [37]:
print(np.mean(scores1))
print(np.mean(scores2))

0.8410235
0.8407802999999999


In [38]:
RESULT_PATH = "./result(developed)/"

In [39]:
pred_ensemble = 0.3 * pred1 + 0.7 * pred2
pred_ensemble

sample_submssion = pd.read_csv('./data/sample_submission.csv')
sample_submssion['problem'] = pred_ensemble.reshape(-1)
sample_submssion.to_csv(RESULT_PATH + "Final_Sub_developed.csv", index = False)
sample_submssion

,user_id,problem
0,30000,0.935817
1,30001,0.385041
2,30002,0.389184
3,30003,0.874505
4,30004,0.991878
...,...,...
14994,44994,0.205993
14995,44995,0.242600
14996,44996,0.682182
14997,44997,1.007793


In [165]:
gn = np.array(list(train_x.columns))
ga = group2[1].get_feature_importance()
gc = pd.DataFrame(ga, gn).reset_index()
gc.columns = ['col','importance']
gc = gc.sort_values(ascending=False, by='importance')
list(gc.col.values[:80])

['err_18_move_3_max',
 'mn2',
 '234',
 'err_18_cnt',
 '360',
 '331',
 'err_15_code',
 '276',
 '391',
 '385',
 '397',
 '394',
 '318',
 'err_13_hour',
 'err_12_move_3_std',
 '150',
 'err_2_hour',
 'err_22_hour',
 'err_4_hour',
 '433',
 '392',
 'err_18_move_7_max',
 'err_9_date',
 'err_20_date',
 'err_26_move_3_min',
 'err_26_date',
 'err_36_code',
 'err_22_date',
 '358',
 'err_11_move_3_std',
 'err_5_qual',
 'err_20_hour',
 'err_0_qual',
 'err_18_move_3_std',
 'err_9_hour',
 '447',
 'err_8_date',
 'err_19_hour',
 'err_31_move_3_std',
 '354',
 '185',
 'err_4_move_3_max',
 'err_17_date',
 'err_23_hour',
 'err_26_move_7_std',
 'err_31_date',
 '357',
 'err_5_hour',
 'err_10_qual',
 'err_24_date',
 'quality_5_max',
 'mn1',
 '79',
 '479',
 '432',
 '186',
 'err_6_hour',
 'err_16_move_7_std',
 'err_14_hour',
 'err_16_move_3_std',
 '375',
 '312',
 'err_21_date',
 'err_6_date',
 'err_16_date',
 'err_33_move_7_std',
 'err_10_hour',
 'err_35_move_3_std',
 'err_1_move_7_std',
 '205',
 '437',
 '242',


In [162]:
gn = np.array(list(train_x.columns))
ga = group2[0].get_feature_importance()
gc = pd.DataFrame(ga, gn).reset_index()
gc.columns = ['col','importance']
gc = gc.sort_values(ascending=False, by='importance')
형빈0 = list(gc.col.values[:100])

gn = np.array(list(train_x.columns))
ga = group2[1].get_feature_importance()
gc = pd.DataFrame(ga, gn).reset_index()
gc.columns = ['col','importance']
gc = gc.sort_values(ascending=False, by='importance')
형빈1 = list(gc.col.values[:100])

gn = np.array(list(train_x.columns))
ga = group2[2].get_feature_importance()
gc = pd.DataFrame(ga, gn).reset_index()
gc.columns = ['col','importance']
gc = gc.sort_values(ascending=False, by='importance')
형빈2 = list(gc.col.values[:100])

gn = np.array(list(train_x.columns))
ga = group2[3].get_feature_importance()
gc = pd.DataFrame(ga, gn).reset_index()
gc.columns = ['col','importance']
gc = gc.sort_values(ascending=False, by='importance')
형빈3 = list(gc.col.values[:100])

gn = np.array(list(train_x.columns))
ga = group2[4].get_feature_importance()
gc = pd.DataFrame(ga, gn).reset_index()
gc.columns = ['col','importance']
gc = gc.sort_values(ascending=False, by='importance')
형빈4 = list(gc.col.values[:100])

gn = np.array(list(train_x.columns))
ga = group1[0].get_feature_importance()
gc = pd.DataFrame(ga, gn).reset_index()
gc.columns = ['col','importance']
gc = gc.sort_values(ascending=False, by='importance')
형빈5 = list(gc.col.values[:100])

gn = np.array(list(train_x.columns))
ga = group1[1].get_feature_importance()
gc = pd.DataFrame(ga, gn).reset_index()
gc.columns = ['col','importance']
gc = gc.sort_values(ascending=False, by='importance')
형빈6 = list(gc.col.values[:100])

gn = np.array(list(train_x.columns))
ga = group1[2].get_feature_importance()
gc = pd.DataFrame(ga, gn).reset_index()
gc.columns = ['col','importance']
gc = gc.sort_values(ascending=False, by='importance')
형빈7 = list(gc.col.values[:100])

gn = np.array(list(train_x.columns))
ga = group1[3].get_feature_importance()
gc = pd.DataFrame(ga, gn).reset_index()
gc.columns = ['col','importance']
gc = gc.sort_values(ascending=False, by='importance')
형빈8 = list(gc.col.values[:100])

gn = np.array(list(train_x.columns))
ga = group1[4].get_feature_importance()
gc = pd.DataFrame(ga, gn).reset_index()
gc.columns = ['col','importance']
gc = gc.sort_values(ascending=False, by='importance')
형빈9 = list(gc.col.values[:100])

set(형빈0) & set(형빈1) & set(형빈2) & set(형빈3) & set(형빈4) & set(형빈5) & set(형빈6) & set(형빈7) & set(형빈8) & set(형빈9)

{'185',
 '186',
 '234',
 '276',
 '331',
 '354',
 '360',
 '385',
 '391',
 '392',
 '394',
 '397',
 '433',
 'err_0_qual',
 'err_10_qual',
 'err_11_move_3_std',
 'err_13_hour',
 'err_14_hour',
 'err_15_code',
 'err_18_cnt',
 'err_18_move_3_max',
 'err_19_hour',
 'err_22_date',
 'err_22_hour',
 'err_23_hour',
 'err_26_date',
 'err_26_move_3_min',
 'err_26_move_7_std',
 'err_2_hour',
 'err_36_code',
 'err_4_hour',
 'err_5_qual',
 'err_8_date',
 'err_9_date',
 'mn1',
 'mn2',
 'quality_5_max'}

In [85]:
# train_x['err_18_move_3_max'] = train_x['err_18_move_3_max'].astype(float)

In [150]:
train_x[234].describe()

count    15000.000000
mean         0.146800
std          0.473073
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         13.000000
Name: 234, dtype: float64

In [152]:
train_x[234].quantile(0.9)

1.0

In [146]:
동인 = train_x.copy()
동인['user_id'] = list(range(10000,25000))

In [155]:
ua = list(동인[동인[234] >= 2].user_id.unique())
len(ua)

282

In [156]:
len(train_problem[train_problem['user_id'].isin(ua)].user_id.unique())

258